## Calculate biomass with GEE JS

Documentation on how to create a microservice with GE engine. In this specific case it 
calculates biomass and biomass density paralelly for all admin 2 regions in the GADM table
by using the map() function. 
Firstly start off by importing the gadm table containing admin level geometries and the 
other 2 tables whrc_carbon and forest_change to help in calculating biomass. 

```javascript
var whrc_carbon = ee.ImageCollection("projects/wri-datalab/WHRC_CARBON"),
    forest_change = ee.Image("projects/wri-datalab/HansenComposite_17"),
    gadm36 = ee.FeatureCollection("projects/wri-datalab/gadm36");
```
   The function below adds a threshold variable for each entry which helps in storing biomass values 
    only upwards of a specific threshold (in this case 10)
```javascript
var addThreshold = function(feature) {
  return feature.set({threshold: 10});
};
var gadm36 = gadm36.map(addThreshold);
```
  The function addBiomass() has a threshold parameter and adds biomass and biomassDensity density 
  properties for all the entries in gadm36 (which consists of admi2 areas). Firstly a mask with tree cover
  is created with help of the forest_change data. 
  The addBiomassSum function gets the biomass_masked property and uses the Reducer function to create 
  a new attribute called biomass. It does so with the feature.set() function. 
  Afterwards, the biomassSum attribute is stored in a variable and used to calculate and add a new attribute
  called biomassDensity which is the biomassSum divided by the area. Finally the geometry of the features is 
  deleted by setting it to null because it isn't needed anymore.
  
  NOTE: feature properties/attributes cannot be printed inside the map() function because it is executed 
  in paralell on different machines. The code has to be written and trusted that it works or debugged in 
  a different way.  
 ```javascript
var addBiomass = function(threshold) {
  var tree_mask = forest_change.select('tree_' + threshold).gt(0);
  var biomass_masked = whrc_carbon.max().multiply(ee.Image.pixelArea().divide(10000)).mask(tree_mask);
  //^^create and apply mask
  var addBiomassSum = function(feature) {
    feature = feature.set({biomass: biomass_masked.reduceRegion({
      reducer: ee.Reducer.sum().unweighted(),
      geometry: feature.geometry(),
      bestEffort: true,
      scale:30 }).get('b1')});              //The actual calculated biomass is in the attribute b1
    var area = feature.geometry().area().divide(100*100);
    var biomassSum = ee.Number(feature.get('biomass'));
    feature = feature.set({biomassDensity: biomassSum.divide(area)})
                      .setGeometry(null); 
    return feature; 
  };
  gadm36 = gadm36.map(addBiomassSum); 
}; 
addBiomass(10);
```
The table needs to be exported, and not printed because it is too large to be printed. 
```javascript
//print(gadm36);  //don't print or else it'll cause scaling errors
Export.table.toCloudStorage({
  collection: gadm36,
  description: 'gadm36Biomass',
  fileFormat: 'CSV'
});
```
